<a href="https://colab.research.google.com/github/DLPY/Regression-Session-2/blob/master/SydneyHousePrices_Ridge_Lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **House Price Prediction based on Postal Code, Number of Bathrooms, Car Parking  and Property Type**

Detail on Data : https://www.kaggle.com/mihirhalai/sydney-house-prices

## 1. Download source data from Github

In [ ]:
!wget https://raw.githubusercontent.com/DLPY/Regression-Session-2/master/Data/SydneyHousePrices.csv

## 2. Import neccssary packages for performing EDA, Lasso and Ridge regression

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

pd.set_option('display.max_colwidth', None)

## 3. Read data into pandas dataframe to perform data analysis, cleaning and transformation

In [ ]:
df = pd.read_csv('SydneyHousePrices.csv')

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
round(df.describe(),2)

**From the above max - It shows the dataset has outliers and it needs to be removed.**

## 4. Choosing predictors and target variables for performing Regression

**Target and Source variables**

*   SellingPrice - Target Variable
*   Bed, Bath, Car, propType, postalCode  - Predictor Variables



In [ ]:
df_new = df[(df['Date']>= '2017-01-01') & (df['Date']< '2020-01-01')].reset_index(drop=True)
df_new = df_new[['postalCode', 'bath', 'car', 'propType' , 'sellPrice']]

In [ ]:
df_new.propType.unique()

In [ ]:
df_new.head(5)

Encoding the categorical variables - Change the text into numbers

In [ ]:
df_new['propType'] = df_new['propType'].astype('category').cat.codes

In [ ]:
df_new.propType.unique()

In [ ]:
df_new.head(5)

In [ ]:
df_new.dtypes

In [ ]:
df_new.count()

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new['car'].fillna(df_new.groupby(['postalCode'])['car'].transform('median'), inplace=True)
#df_new['bed'].fillna(df_new.groupby(['postalCode', 'propType'])['bed'].transform('median'), inplace=True)

In [ ]:
df_new.isnull().sum()

## 5. Remove outliers in the data

In [ ]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
#df_no_outlier_bed=remove_outlier(df_new, 'bed').reset_index(drop=True)
df_no_outlier_bath=remove_outlier(df_new, 'bath').reset_index(drop=True)
df_no_outlier_car=remove_outlier(df_no_outlier_bath, 'car').reset_index(drop=True)
df_no_outlier=remove_outlier(df_no_outlier_car, 'sellPrice').reset_index(drop=True)

In [ ]:
df_no_outlier

In [ ]:
corr = df_no_outlier.corr()

In [ ]:
corr

## 6. Correlation heatmap with the mask and correct aspect ratio

In [ ]:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(10, 250, as_cmap=True)
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, cmap=cmap,
            square=True, linewidths=.2, cbar_kws={'shrink': .5}, ax=ax, annot=True)

## 7. Split Target and Predictor Variables to different dataframes

In [ ]:
X = df_no_outlier.iloc[:,:-1]
y = df_no_outlier.iloc[:,4]

In [ ]:
X.head(5)

In [ ]:
y.head(5)

Convert dataframes to values to feed into Model

In [ ]:
X = X.values
y = y.values

In [ ]:
print('Number of records and predictor variables:', X.shape)
print('Number of records and target variable:', y.shape)

## 8. Split dataset into the training and test using train_set_split: 90% - train and 10% - test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=23)

In [ ]:
print('Training Data:', X_train.shape, y_train.shape)
print('Testing Data:', X_test.shape, y_test.shape)

## 9. Train, Test and Predict using ridge and lasso regression model

Create an object called RidgeRegression in the regression class with alpha 0.01

In [ ]:
ridgeregressor = Ridge(alpha=0.01)

Fit the linear regression model to the training set. We use the fit method the arguments of the fit method will be training sets

In [ ]:
ridgeregressor.fit(X_train, y_train)

Regression Coefficients

In [ ]:
print('Coefficients: ', ridgeregressor.coef_)

In [ ]:
column_names = ['postalCode', 'bath', 'car', 'propType' ]
coefficient_df = pd.DataFrame(ridgeregressor.coef_).T
coefficient_df.columns = column_names
coefficient_df

Predicting the Test set results

In [ ]:
y_pred= ridgeregressor.predict(X_test)

## 10. Evaluation metrics - How to Calculate R-Square and RMSE

In [ ]:
coefficient_of_dermination = r2_score(y_test,y_pred)
print('R-squared:',coefficient_of_dermination)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse))

In [ ]:
# Displaying Results and Difference in Table 
res = pd.DataFrame(y_pred, y_test)
res = res.reset_index()
res.columns = ['Price', 'Prediction']
res['Prediction'] = round(res['Prediction'],0)
res['Difference'] = res['Price'] - res['Prediction']
res.head(5)

Create an object called RidgeRegression in the regression class with alpha 100

In [ ]:
ridgeregressor100 = Ridge(alpha=100)
ridgeregressor100.fit(X_train, y_train)
print('Coefficients: ', ridgeregressor100.coef_)

In [ ]:
column_names = ['postalCode', 'bath', 'car', 'propType' ]
coefficient_df = pd.DataFrame(ridgeregressor100.coef_).T
coefficient_df.columns = column_names
coefficient_df

In [ ]:
y_pred= ridgeregressor100.predict(X_test)

coefficient_of_dermination = r2_score(y_test, y_pred)
print('R-squared:',coefficient_of_dermination)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse))

In [ ]:
# Displaying Results and Difference in Table 
res = pd.DataFrame(y_pred, y_test)
res = res.reset_index()
res.columns = ['Price', 'Prediction']
res['Prediction'] = round(res['Prediction'],0)
res['Difference'] = res['Price'] - res['Prediction']
res.head(5)

Create an object called Lasso in the regression class with alpha 0.01

In [ ]:
lassoregressor = Lasso(alpha=0.01, max_iter=10e6)
lassoregressor.fit(X_train, y_train)
print('Coefficients: ', lassoregressor.coef_)

In [ ]:
column_names = ['postalCode', 'bath', 'car', 'propType' ]
coefficient_df = pd.DataFrame(lassoregressor.coef_).T
coefficient_df.columns = column_names
coefficient_df

In [ ]:
coeff_used = np.sum(lassoregressor.coef_!=0)
coeff_used

In [ ]:
y_pred= lassoregressor.predict(X_test)

coefficient_of_dermination = r2_score(y_test, y_pred)
print('R-squared:',coefficient_of_dermination)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse))

Create an object called Lasso in the regression class with alpha 100

In [ ]:
lassoregressor100 = Lasso(alpha=100, max_iter=10e6)
lassoregressor100.fit(X_train, y_train)
print('Coefficients: ', lassoregressor100.coef_)

In [ ]:
column_names = ['postalCode', 'bath', 'car', 'propType' ]
coefficient_df = pd.DataFrame(lassoregressor100.coef_).T
coefficient_df.columns = column_names
coefficient_df

In [ ]:
y_pred= lassoregressor100.predict(X_test)

coefficient_of_dermination = r2_score(y_test, y_pred)
print('R-squared:',coefficient_of_dermination)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse))

## 12. Regression on Full data using OLS model

In [ ]:
Regression = sm.OLS(endog = y, exog = X).fit()

In [ ]:
print(Regression.summary())

###To read more on lasso and ridge regression.

---
https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b